In [1]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [2]:
html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

print(soup.prettify())


<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    Elsie
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>



In [9]:
url = 'http://localhost:5500/2025_Spring_Data-Management/week_02/bs4_doc.html'
import requests
res = requests.get(url)
print(res)

<Response [200]>


In [11]:
print(res.headers['content-type'])
print(res.encoding)
print(res.text)

text/html; charset=UTF-8
UTF-8
<html><head><title>The Dormouse's story</title><!-- Code injected by live-server -->
<script>
	// <![CDATA[  <-- For SVG support
	if ('WebSocket' in window) {
		(function () {
			function refreshCSS() {
				var sheets = [].slice.call(document.getElementsByTagName("link"));
				var head = document.getElementsByTagName("head")[0];
				for (var i = 0; i < sheets.length; ++i) {
					var elem = sheets[i];
					var parent = elem.parentElement || head;
					parent.removeChild(elem);
					var rel = elem.rel;
					if (elem.href && typeof rel != "string" || rel.length == 0 || rel.toLowerCase() == "stylesheet") {
						var url = elem.href.replace(/(&|\?)_cacheOverride=\d+/, '');
						elem.href = url + (url.indexOf('?') >= 0 ? '&' : '?') + '_cacheOverride=' + (new Date().valueOf());
					}
					parent.appendChild(elem);
				}
			}
			var protocol = window.location.protocol === 'http:' ? 'ws://' : 'wss://';
			var address = protocol + window.location.host + window.lo

원본 HTML과 res.text의 내용이 많이 다른 이유는 다음과 같습니다.

1. Live Server의 코드 삽입
res.text의 응답에 "<!-- Code injected by live-server -->"라는 주석과 함께 JavaScript 코드가 포함되어 있습니다.
이는 **Live Server (VS Code의 live-server 확장)**가 자동으로 페이지를 새로고침하기 위해 삽입하는 코드입니다.
브라우저에서 F12 (개발자 도구)로 확인한 원본 HTML에는 이 코드가 없지만, requests.get(url)로 가져온 데이터에는 포함될 수 있습니다.
2. JavaScript 실행 여부
requests.get(url)은 정적 HTML만 가져오기 때문에, JavaScript가 동적으로 변경한 HTML을 반영하지 않습니다.
브라우저에서 F12로 확인한 HTML은 JavaScript가 실행된 후의 결과이므로 차이가 발생합니다.
예를 들어, 특정 JavaScript가 실행되어 DOM 요소를 추가하거나 변경했을 경우, 브라우저에서는 이를 반영하지만 requests.get(url)의 응답에서는 볼 수 없습니다.
3. 실제 서버의 응답과 브라우저 렌더링의 차이
requests.get(url)로 가져온 HTML은 서버에서 전송된 원본 HTML이며, 브라우저는 이를 렌더링하면서 추가적인 JavaScript 및 CSS 로직을 실행하여 최종적으로 표시합니다.
개발자 도구에서 본 HTML은 JavaScript가 실행된 후의 결과이므로, 서버에서 직접 가져온 HTML과 차이가 발생할 수 있습니다.

F12의 Elements 패널과 Network 패널의 Response 탭에서 보이는 코드의 차이점은 다음과 같습니다.

1. Elements 패널 (DOM Tree)
실제 브라우저에서 렌더링된 후의 HTML을 보여줍니다.
JavaScript가 실행된 후의 최종적인 결과를 반영합니다.
예를 들어, Live Server에서 자동으로 추가한 코드가 실행된 후의 상태나, JavaScript가 동적으로 생성한 요소가 포함될 수 있습니다.
2. Network 패널 → Response 탭
서버에서 초기 응답으로 받은 HTML 소스 코드를 그대로 보여줍니다.
JavaScript가 실행되기 전의 원본 HTML입니다.
즉, requests.get(url).text와 동일한 내용을 확인할 수 있습니다.
3. 차이점 분석
Network Response 코드: 서버에서 처음 받은 원본 HTML (JavaScript 실행 전)
Elements 패널 코드: JavaScript가 실행된 후 최종적으로 브라우저가 해석한 HTML (DOM Tree)
예를 들어, Live Server가 HTML에 추가한 WebSocket 관련 스크립트(<!-- Code injected by live-server -->)는 Network 패널의 Response 탭에서도 확인할 수 있지만, JavaScript 실행 후 동적으로 변경된 내용은 Elements 패널에서만 보일 수 있습니다.

즉,

Network Response 코드는 서버에서 받은 "정적인 HTML"
Elements 패널 코드는 JavaScript 실행 후의 "동적인 HTML"을 반영한 결과


만약 JavaScript가 실행된 후의 최종 HTML을 가져오고 싶다면, requests 대신 Selenium을 사용해야 합니다.

In [13]:
from selenium import webdriver

url = 'http://localhost:5500/2025_Spring_Data-Management/week_02/bs4_doc.html'

# 브라우저 실행 (Chrome 기준)
driver = webdriver.Chrome()
driver.get(url)

# JavaScript 실행 후 최종 HTML 가져오기
html = driver.page_source
print(html)

driver.quit()


<html><head><title>The Dormouse's story</title><!-- Code injected by live-server -->
<script>
	// <![CDATA[  <-- For SVG support
	if ('WebSocket' in window) {
		(function () {
			function refreshCSS() {
				var sheets = [].slice.call(document.getElementsByTagName("link"));
				var head = document.getElementsByTagName("head")[0];
				for (var i = 0; i < sheets.length; ++i) {
					var elem = sheets[i];
					var parent = elem.parentElement || head;
					parent.removeChild(elem);
					var rel = elem.rel;
					if (elem.href && typeof rel != "string" || rel.length == 0 || rel.toLowerCase() == "stylesheet") {
						var url = elem.href.replace(/(&|\?)_cacheOverride=\d+/, '');
						elem.href = url + (url.indexOf('?') >= 0 ? '&' : '?') + '_cacheOverride=' + (new Date().valueOf());
					}
					parent.appendChild(elem);
				}
			}
			var protocol = window.location.protocol === 'http:' ? 'ws://' : 'wss://';
			var address = protocol + window.location.host + window.location.pathname + '/ws';
			var

### 2 개 코듸의 차이점을 알고 싶다면 아주 미묘한 차이인데.. 챗에게 물어봐

1. socket.onmessage 핸들러 포함 여부
첫 번째 코드에는 다음 부분이 없음:

javascript
socket.onmessage = function (msg) {
    if (msg.data == 'reload') window.location.reload();

- 두 번째 코드에는 socket.onmessage 이벤트 리스너가 포함되어 있습니다.
- 이 부분은 Live Server의 WebSocket이 특정 메시지를 받았을 때 페이지를 자동으로 새로고침하도록 하는 기능입니다.
- 첫 번째 코드에서는 이 부분이 생략되어 있어, Live Server가 자동 새로고침을 수행하지 않을 수 있습니다.

2. </body></html> 태그의 유무
- 첫 번째 코드는 </body></html> 태그가 없음 → HTML 문서가 정상적으로 닫히지 않음
- 두 번째 코드는 </body></html> 태그가 존재 → 정상적인 HTML 구조
- 첫 번째 코드가 출력 중 일부가 잘린(truncated) 것인지, 원래부터 불완전한 상태인지는 추가 확인이 필요함.

3. Output is truncated 메시지
- 두 코드의 마지막에는 공통적으로 Output is truncated 메시지가 포함되어 있음.

- 이는 실행 환경(예: Jupyter Notebook, 웹 기반 출력 창)에서 너무 긴 HTML 데이터를 한 번에 출력할 수 없어서 출력 내용을 자른(truncated) 상태일 가능성이 높음.
따라서 실제 차이점을 확인하려면, 전체 HTML을 파일로 저장하여 직접 열어보거나, print() 대신 파일 쓰기(write())로 확인하는 것이 필요함.

#### 결론
> 첫 번째 코드
>> socket.onmessage가 없음 (Live Server의 자동 새로고침 기능이 비활성화되었을 가능성)
>> </body></html>가 없음 (출력이 일부 잘린 것일 수도 있음)


> 두 번째 코드
>> socket.onmessage 포함 (Live Server의 자동 새로고침 기능 활성화)
>> </body></html> 태그가 존재 (정상적인 HTML 구조)
→ 첫 번째 코드가 불완전한 상태이거나, 일부 출력이 잘린 상태일 가능성이 높음.
→ Network 패널에서 확인한 Response를 파일로 저장해 비교하는 것이 가장 확실한 방법.

In [18]:
import requests
from bs4 import BeautifulSoup

# 요청할 URL
url = 'http://localhost:5500/2025_Spring_Data-Management/week_02/bs4_doc.html'

# HTTP GET 요청 보내기
res = requests.get(url)

# 응답 HTML 파싱
soup = BeautifulSoup(res.text, 'html.parser')

# 파싱된 HTML 출력 (보기 좋게 정리)
print(soup.prettify())


<html>
 <head>
  <title>
   The Dormouse's story
  </title>
  <!-- Code injected by live-server -->
  <script>
   // <![CDATA[  <-- For SVG support
	if ('WebSocket' in window) {
		(function () {
			function refreshCSS() {
				var sheets = [].slice.call(document.getElementsByTagName("link"));
				var head = document.getElementsByTagName("head")[0];
				for (var i = 0; i < sheets.length; ++i) {
					var elem = sheets[i];
					var parent = elem.parentElement || head;
					parent.removeChild(elem);
					var rel = elem.rel;
					if (elem.href && typeof rel != "string" || rel.length == 0 || rel.toLowerCase() == "stylesheet") {
						var url = elem.href.replace(/(&|\?)_cacheOverride=\d+/, '');
						elem.href = url + (url.indexOf('?') >= 0 ? '&' : '?') + '_cacheOverride=' + (new Date().valueOf());
					}
					parent.appendChild(elem);
				}
			}
			var protocol = window.location.protocol === 'http:' ? 'ws://' : 'wss://';
			var address = protocol + window.location.host + window.location.pathn


## 방법 1. 단순한 Python HTTP 서버 사용
Live Server 대신 Python 내장 웹 서버를 실행하면 불필요한 WebSocket 코드가 삽입되지 않음.

1) Python HTTP 서버 실행
터미널에서 HTML 파일이 있는 폴더에서 실행:

(.venv) D:\modern_web\2025_Spring_Data-Management\week_02>python -m http.server 5500  

In [21]:
url = 'http://localhost:5500/bs4_doc.html'

res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    Elsie
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>



로컬 HTML을 직접 open()으로 읽으면 100% 원본 HTML 그대로 가져올 수 있음.

In [22]:
with open("bs4_doc.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, 'html.parser')

print(soup.prettify())


<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    Elsie
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>

